In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle
from copy import deepcopy
import random

# disable warnings to ignore overflow error
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
# parameters for CSTR
T_0_cstr = 300
V_cstr = 1
k_0_cstr = 8.46*(np.power(10,6))
C_p_cstr = 0.231
rho_L_cstr = 1000
Q_s_cstr = 0.0
F_cstr = 5
E_cstr = 5*(np.power(10,4))
delta_H_cstr = -1.15*(np.power(10,4))
R_cstr = 8.314
C_A0s_cstr = 4

t_final_cstr = 0.005
t_step_cstr = 1e-4

# parameters for Batch
V_batch = 1
k_0_batch = 8.46*(np.power(10,7))
C_p_batch = 0.231
rho_L_batch = 1000
Q_s_batch = 0
E_batch = 5*(np.power(10,4))
delta_H_batch = -1.15*(np.power(10,4))
R_batch = 8.314

t_final_batch = 0.05
t_step_batch = 1e-4

# parameters for neural networks
num_step = 10
num_dims = 4

# parameters for transfer learning
seed = 0
plot = True
rng = np.random.RandomState(seed)
innerepochs = 1 # number of epochs of each inner SGD
niterations = 1000 # number of outer updates; each iteration we sample one task and update on it
ntrain = 32 # size of training minibatches (K)
eval_step = 50 # evaluation step
threshold = 10 # threshold to check data correctness

In [ ]:
def generate_new(x):
    return x + x * np.random.uniform(-10, 10)

def generate_new_small(x):
    return x + x * np.random.uniform(-0.05, 0.05)

def Batch_simulation(V, k_0, E, R, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating Batch using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = -k_0 * np.exp(-E / (R * T)) * C_A
        dTdt = - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 50 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 5 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def to_tensor(x):
    return tf.convert_to_tensor(x, dtype=tf.float32)

def train_on_batch(x, y, model, optimizer):
    x = to_tensor(x)
    y = to_tensor(y)

    with tf.GradientTape() as tape:
        YHat = model(x)
        loss = mse_loss_fn(y, YHat)
        grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

def predict(x, model):
    x = to_tensor(x)
    return model(x).numpy()

def compute_loss(x, y, model):
    return np.square(predict(x, model) - y).mean()

In [ ]:
def gen_cstr(F, V, C_A0s, k_0, E, R, T_0, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step):
    isCorrect = False
    while isCorrect == False:
        T_0_new = generate_new(T_0)
        V_new = generate_new(V)
        F_new = generate_new(F)
        C_A0s_new = generate_new(C_A0s)
        Q_s_new = generate_new(Q_s)
        rho_L_new = generate_new_small(rho_L)
        C_p_new = generate_new_small(C_p)
        k_0_new = generate_new_small(k_0)
        E_new = generate_new_small(E)
        delta_H_new = generate_new_small(delta_H)

        # generating inputs and initial states for CSTR
        u1_list = np.linspace(-3.5, 3.5, 4, endpoint=True)
        u2_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 20, endpoint=True)
        CA_initial = np.linspace(0, 6, 20, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u1_list:
            C_A0 = u1 + C_A0s_new
            for u2 in u2_list:
                Q = u2 + Q_s_new
                for C_A_initial, T_initial in x_deviation:

                    C_A_list, T_list = CSTR_simulation(F_new, V_new, C_A0, k_0_new, E_new, R, T_0_new, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                    if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                        CA0_input.append(u1)
                        Q_input.append(u2)
                        CA_input.append(C_A_initial)
                        T_input.append(T_initial)

                        CA_output.append(C_A_list)
                        T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of CSTR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [ ]:
def gen_batch(V, k_0, E, R, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step):
    isCorrect = False
    while isCorrect == False:
        V_new = generate_new(V)
        Q_s_new = generate_new(Q_s)
        rho_L_new = generate_new_small(rho_L)
        C_p_new = generate_new_small(C_p)
        k_0_new = generate_new(k_0)
        E_new = generate_new_small(E)
        delta_H_new = generate_new_small(delta_H)

        # generating inputs and initial states for Batch
        u_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 40, endpoint=True)
        CA_initial = np.linspace(0, 6, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u_list:
            Q = u1 + Q_s_new
            for C_A_initial, T_initial in x_deviation:

                C_A_list, T_list = Batch_simulation(V_new, k_0_new, E_new, R, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Q_input.append(u1)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, Q_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of Batch: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [ ]:
class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Necessary to create the model's state.
# The model doesn't have a state until it's called at least once.
_ = model(tf.zeros((ntrain, num_step, num_dims)))

optimizer = tf.keras.optimizers.Adam()
mse_loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
# generate testing cstr
isOverflow = True
while isOverflow == True:
    try:
        x_test_cstr, y_test_cstr = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_cstr), size=1)
xtest_plot_1shot_cstr = x_test_cstr[sample_idx]
ytest_plot_1shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=5)
xtest_plot_5shot_cstr = x_test_cstr[sample_idx]
ytest_plot_5shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=10)
xtest_plot_10shot_cstr = x_test_cstr[sample_idx]
ytest_plot_10shot_cstr = y_test_cstr[sample_idx]

# generate testing batch
isOverflow = True
while isOverflow == True:
    try:
        x_test_batch, y_test_batch = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_batch), size=1)
xtest_plot_1shot_batch = x_test_batch[sample_idx]
ytest_plot_1shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=5)
xtest_plot_5shot_batch = x_test_batch[sample_idx]
ytest_plot_5shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=10)
xtest_plot_10shot_batch = x_test_batch[sample_idx]
ytest_plot_10shot_batch = y_test_batch[sample_idx]

Number of training samples of CSTR:  6064
Number of training samples of Batch:  6222


In [ ]:
training_loss = np.zeros(niterations)
test_loss_1shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_batch = np.zeros(int(niterations/eval_step+1))
count = 0
countBatch = 0
countCSTR = 0

reactor = 0

# transfer
for iteration in range(niterations):

    print("Begin iteration ", iteration)

    # generate task
    isOverflow = True
    while isOverflow == True:
        try:
            # switch to another reactor, 0 for cstr and 1 for batch
            # reactor = random.randint(0,1)
            if reactor == 0:
                x_all, y_all = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step)
                countCSTR = countCSTR + 1
                reactor = 1
            elif reactor == 1:
                x_all, y_all = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step)
                countBatch = countBatch + 1
                reactor = 0
            isOverflow = False
        except ValueError:
            pass

    inds = rng.permutation(len(x_all))

    # for some k number of iterations perform optimization on the task
    for k in range(innerepochs):

        for start in range(0, len(x_all), ntrain):
            mbinds = inds[start:start+ntrain]
            train_on_batch(x_all[mbinds], y_all[mbinds], model, optimizer)

    training_loss[iteration] = compute_loss(x_all, y_all, model)
    print("Training loss: ", training_loss[iteration])

    # reset weights and terminate for NaN training loss
    if np.isnan(training_loss[iteration]) == True:
        model.set_weights(weights_before)
        break

    # begin evaluation
    if plot and iteration==0 or (iteration+1) % eval_step == 0:
        # 1 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_cstr, ytest_plot_1shot_cstr, model, optimizer)

        test_loss_1shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 1-shot CSTR: ", test_loss_1shot_cstr[count])
        model.set_weights(weights_before)

        # 5 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_cstr, ytest_plot_5shot_cstr, model, optimizer)

        test_loss_5shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 5-shots CSTR: ", test_loss_5shot_cstr[count])
        model.set_weights(weights_before)

        # 10 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_cstr, ytest_plot_10shot_cstr, model, optimizer)

        test_loss_10shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 10-shots CSTR: ", test_loss_10shot_cstr[count])
        model.set_weights(weights_before)

        # 1 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_batch, ytest_plot_1shot_batch, model, optimizer)

        test_loss_1shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 1-shot Batch: ", test_loss_1shot_batch[count])
        model.set_weights(weights_before)

        # 5 shot learning for bacth
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_batch, ytest_plot_5shot_batch, model, optimizer)

        test_loss_5shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 5-shots Batch: ", test_loss_5shot_batch[count])
        model.set_weights(weights_before)

        # 10 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_batch, ytest_plot_10shot_batch, model, optimizer)

        test_loss_10shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 10-shots Batch: ", test_loss_10shot_batch[count])
        model.set_weights(weights_before)

        count = count + 1

    if (iteration+1) % 100 == 0:
        filename = 'model_transfer_cstr_batch.sav'
        pickle.dump(model, open(filename, 'wb'))

Begin iteration  0
Number of training samples of CSTR:  6080
Training loss:  0.017386654764555615
Test loss for 1-shot CSTR:  0.044146530096058866
Test loss for 5-shots CSTR:  0.03997674606747463
Test loss for 10-shots CSTR:  0.09367811007897732
Test loss for 1-shot Batch:  1.352904437112943
Test loss for 5-shots Batch:  1.3016940413287217
Test loss for 10-shots Batch:  1.615023331562522
Begin iteration  1
Number of training samples of Batch:  6118
Training loss:  0.17704099590880656
Begin iteration  2
Number of training samples of CSTR:  6400
Training loss:  0.014614902110582534
Begin iteration  3
Number of training samples of Batch:  6240
Training loss:  0.02123180063694383
Begin iteration  4
Number of training samples of CSTR:  6400
Training loss:  0.02121950271999942
Begin iteration  5
Number of training samples of Batch:  6194
Training loss:  0.13233196284689422
Begin iteration  6
Number of training samples of CSTR:  6400
Training loss:  0.022908235858818102
Begin iteration  7
Num

Number of training samples of Batch:  6240
Training loss:  0.024952216307032447
Begin iteration  78
Number of training samples of CSTR:  6160
Training loss:  0.022792420372307626
Begin iteration  79
Number of training samples of Batch:  6240
Training loss:  0.019580809950738645
Begin iteration  80
Number of training samples of CSTR:  6400
Training loss:  0.026955724105636385
Begin iteration  81
Number of training samples of Batch:  6240
Training loss:  0.04640032621047095
Begin iteration  82
Number of training samples of CSTR:  6080
Training loss:  0.031218395742153317
Begin iteration  83
Number of training samples of Batch:  6216
Training loss:  0.027123037032195086
Begin iteration  84
Number of training samples of CSTR:  6136
Training loss:  0.031440650420955465
Begin iteration  85
Number of training samples of Batch:  6240
Training loss:  0.012474667392228756
Begin iteration  86
Number of training samples of CSTR:  6400
Training loss:  0.028009948990967516
Begin iteration  87
Number

Training loss:  0.02699436895638616
Begin iteration  154
Number of training samples of CSTR:  6168
Training loss:  0.03420981731218733
Begin iteration  155
Number of training samples of Batch:  6240
Training loss:  0.017320812417735844
Begin iteration  156
Number of training samples of CSTR:  6360
Training loss:  0.029419692108429752
Begin iteration  157
Number of training samples of Batch:  6240
Training loss:  0.01735140179094196
Begin iteration  158
Number of training samples of CSTR:  6400
Training loss:  0.03935327847596962
Begin iteration  159
Number of training samples of Batch:  5953
Training loss:  0.014396918574866477
Begin iteration  160
Number of training samples of CSTR:  6400
Training loss:  0.038045530277634715
Begin iteration  161
Number of training samples of Batch:  6240
Training loss:  0.028866982947665855
Begin iteration  162
Number of training samples of CSTR:  6400
Training loss:  0.03360353081903869
Begin iteration  163
Number of training samples of Batch:  6181


Training loss:  0.03139981373128829
Begin iteration  233
Number of training samples of Batch:  6240
Training loss:  0.020189159414493965
Begin iteration  234
Number of training samples of CSTR:  6400
Training loss:  0.03554189148675284
Begin iteration  235
Number of training samples of Batch:  5939
Training loss:  0.013738514525527863
Begin iteration  236
Number of training samples of CSTR:  6384
Training loss:  0.03839230370547113
Begin iteration  237
Number of training samples of Batch:  6240
Training loss:  0.024494729718310536
Begin iteration  238
Number of training samples of CSTR:  6384
Training loss:  0.04381407655559515
Begin iteration  239
Number of training samples of Batch:  5724
Training loss:  0.016775330308600043
Begin iteration  240
Number of training samples of CSTR:  6129
Training loss:  0.04496292420651322
Begin iteration  241
Number of training samples of Batch:  6240
Training loss:  0.03940548878601034
Begin iteration  242
Number of training samples of CSTR:  6400
T

Number of training samples of Batch:  5859
Training loss:  0.011316068073408484
Begin iteration  310
Number of training samples of CSTR:  6400
Training loss:  0.04162194541060589
Begin iteration  311
Number of training samples of Batch:  6239
Training loss:  0.023676541111041966
Begin iteration  312
Number of training samples of CSTR:  6400
Training loss:  0.05147051430385989
Begin iteration  313
Number of training samples of Batch:  6216
Training loss:  0.010887002722987673
Begin iteration  314
Number of training samples of CSTR:  6064
Training loss:  0.0429779501703754
Begin iteration  315
Number of training samples of Batch:  5876
Training loss:  0.016868461339387058
Begin iteration  316
Number of training samples of CSTR:  6400
Training loss:  0.05248936578880545
Begin iteration  317
Number of training samples of Batch:  6240
Training loss:  0.024496238467461774
Begin iteration  318
Number of training samples of CSTR:  6276
Training loss:  0.046274791069034746
Begin iteration  319


...

Number of training samples of Batch:  6240
Training loss:  0.03295064148718193
Begin iteration  622
Number of training samples of CSTR:  6399
Training loss:  0.04342995897437792
Begin iteration  623
Number of training samples of Batch:  6166
Training loss:  0.014794273238752991
Begin iteration  624
Number of training samples of CSTR:  6157
Training loss:  0.06197677852882324
Begin iteration  625
Number of training samples of Batch:  5922
Training loss:  0.016840306941841146
Begin iteration  626
Number of training samples of CSTR:  6320
Training loss:  0.04521354070832554
Begin iteration  627
Number of training samples of Batch:  6218
Training loss:  0.02192127630589484
Begin iteration  628
Number of training samples of CSTR:  6384
Training loss:  0.08676494112049282
Begin iteration  629
Number of training samples of Batch:  6240
Training loss:  0.02960237163376557
Begin iteration  630
Number of training samples of CSTR:  6400
Training loss:  0.042480050926802335
Begin iteration  631
Nu

Test loss for 5-shots CSTR:  0.9988377078720024
Test loss for 10-shots CSTR:  1.16752832142519
Test loss for 1-shot Batch:  0.07739906737657667
Test loss for 5-shots Batch:  0.04179391924486136
Test loss for 10-shots Batch:  0.07399429497762414
Begin iteration  700
Number of training samples of CSTR:  6320
Training loss:  0.053052309117360454
Begin iteration  701
Number of training samples of Batch:  6240
Training loss:  0.01964041144524294
Begin iteration  702
Number of training samples of CSTR:  6400
Training loss:  0.09755719945106156
Begin iteration  703
Number of training samples of Batch:  6240
Training loss:  0.03826315019086713
Begin iteration  704
Number of training samples of CSTR:  6400
Training loss:  0.04476360406307474
Begin iteration  705
Number of training samples of Batch:  6240
Training loss:  0.019192354070003546
Begin iteration  706
Number of training samples of CSTR:  6232
Training loss:  0.05180410753176839
Begin iteration  707
Number of training samples of Batch:

Number of training samples of Batch:  6224
Training loss:  0.012665333571335846
Begin iteration  778
Number of training samples of CSTR:  6400
Training loss:  0.03603117321162707
Begin iteration  779
Number of training samples of Batch:  6240
Training loss:  0.026249235172699146
Begin iteration  780
Number of training samples of CSTR:  6080
Training loss:  0.05442508824457816
Begin iteration  781
Number of training samples of Batch:  6240
Training loss:  0.014807913155876486
Begin iteration  782
Number of training samples of CSTR:  6080
Training loss:  0.050030865843728206
Begin iteration  783
Number of training samples of Batch:  6178
Training loss:  0.013033574945571454
Begin iteration  784
Number of training samples of CSTR:  6400
Training loss:  0.05736471146046197
Begin iteration  785
Number of training samples of Batch:  6240
Training loss:  0.024725846975809804
Begin iteration  786
Number of training samples of CSTR:  6399
Training loss:  0.042614807944576796
Begin iteration  78

Training loss:  0.01762369682326409
Begin iteration  854
Number of training samples of CSTR:  6364
Training loss:  0.0508792015359411
Begin iteration  855
Number of training samples of Batch:  6221
Training loss:  0.011898220181810533
Begin iteration  856
Number of training samples of CSTR:  6371
Training loss:  0.05060989786426713
Begin iteration  857
Number of training samples of Batch:  6240
Training loss:  0.02365183458037562
Begin iteration  858
Number of training samples of CSTR:  6325
Training loss:  0.04043795374057349
Begin iteration  859
Number of training samples of Batch:  6239
Training loss:  0.0167410804906805
Begin iteration  860
Number of training samples of CSTR:  6400
Training loss:  0.06515120129444375
Begin iteration  861
Number of training samples of Batch:  6240
Training loss:  0.02535594351941458
Begin iteration  862
Number of training samples of CSTR:  6400
Training loss:  0.03801466224864284
Begin iteration  863
Number of training samples of Batch:  6240
Traini

Number of training samples of Batch:  6224
Training loss:  0.021685397218600175
Begin iteration  934
Number of training samples of CSTR:  6400
Training loss:  0.059023214789185145
Begin iteration  935
Number of training samples of Batch:  5932
Training loss:  0.01223430590758668
Begin iteration  936
Number of training samples of CSTR:  6400
Training loss:  0.061550220637505544
Begin iteration  937
Number of training samples of Batch:  5975
Training loss:  0.014332134781051744
Begin iteration  938
Number of training samples of CSTR:  6400
Training loss:  0.05479482949338701
Begin iteration  939
Number of training samples of Batch:  6231
Training loss:  0.01700857338498598
Begin iteration  940
Number of training samples of CSTR:  6320
Training loss:  0.05783770149678841
Begin iteration  941
Number of training samples of Batch:  6240
Training loss:  0.018433514984442526
Begin iteration  942
Number of training samples of CSTR:  6240
Training loss:  0.04433252823463146
Begin iteration  943


In [ ]:
filename = 'model_transfer_cstr_batch.sav'
pickle.dump(model, open(filename, 'wb'))
# model = pickle.load(open(filename, 'rb'))

In [ ]:
np.savetxt("training_transfer_cstr_batch.txt", training_loss, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_transfer_cstr_cstr_batch.txt", test_loss_1shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_transfer_cstr_cstr_batch.txt", test_loss_5shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_transfer_cstr_cstr_batch.txt", test_loss_10shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_transfer_batch_cstr_batch.txt", test_loss_1shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_transfer_batch_cstr_batch.txt", test_loss_5shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_transfer_batch_cstr_batch.txt", test_loss_10shot_batch, fmt='%f', delimiter=" ")